# Exploration notebook

In [1]:
from pycomfort.files import *
from pathlib import Path
from collections import OrderedDict
import polars as pl
from nam.config import Locations
from nam.datasets import DatasetNAM, read_tsv

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
%load_ext autoreload
%autoreload

In [4]:
from polars import Config
Config.set_fmt_str_lengths(5000)

polars.config.Config

# Setting things up

In [5]:
from langchainplus_sdk import LangChainPlusClient
client = LangChainPlusClient()
client

LangChainPlusClient (API URL: http://localhost:1984)

In [6]:
base = Path("..").absolute().resolve()
print(base)
locations = Locations(base)
datasets = OrderedDict ((d.name, DatasetNAM(d)) for d in dirs(locations.datasets) if (d / "models.tsv").exists())
datasets

/home/antonkulaga/sources/non-animal-models


OrderedDict([('respiratory', <nam.datasets.DatasetNAM at 0x7f026b397c40>),
             ('immuno-oncology', <nam.datasets.DatasetNAM at 0x7f02ac277b80>),
             ('neurodegenerative',
              <nam.datasets.DatasetNAM at 0x7f02ac277be0>),
             ('ATMP', <nam.datasets.DatasetNAM at 0x7f02ac276ef0>),
             ('breast-cancer', <nam.datasets.DatasetNAM at 0x7f02ac277100>),
             ('CVD', <nam.datasets.DatasetNAM at 0x7f026f943730>)])

# Exploring dataset

In [7]:
datasets

OrderedDict([('respiratory', <nam.datasets.DatasetNAM at 0x7f026b397c40>),
             ('immuno-oncology', <nam.datasets.DatasetNAM at 0x7f02ac277b80>),
             ('neurodegenerative',
              <nam.datasets.DatasetNAM at 0x7f02ac277be0>),
             ('ATMP', <nam.datasets.DatasetNAM at 0x7f02ac276ef0>),
             ('breast-cancer', <nam.datasets.DatasetNAM at 0x7f02ac277100>),
             ('CVD', <nam.datasets.DatasetNAM at 0x7f026f943730>)])

# ATMP

In [8]:
atmp = datasets["ATMP"]
atmp.models
atmp.doi_cols

['Link to Abstract (DOI)']

In [9]:
dropdowns_atmp = atmp.dropdowns
dropdowns_atmp

Disease area,Study / Disease Feature,ATMP type,Category,Type,Cells,ATMP Input,Cell culture type,Cell culture dimensions,3D type,Model Cell Source 1,Model Cell Source 2,Model Cell Source 3,Biological endpoints,Throughput,Potential,Potential 2,Relevance,Status,Content,Predictive,Year
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64
"""AIDS""","""3D modeling""","""Cell therapy""","""In silico""","""Cell free""","""Immortalised cells""","""Adenovirus / Non-small cell lung cancer""","""Cell free culture""","""2D""","""n/a""","""A459""","""CD3+ T lymphocytes""","""CD34+ Hematopoietic Stem and Progenitor Cells""","""Activation markers""","""High""","""Model development / experimental""","""n/a""","""Direct""","""In research use""","""High""","""n/a""",2014
"""Allotransplantation""","""Alloreactivity study ""","""Gene therapy""","""In vitro""","""Cells""","""n/a""","""Adenovirus / PSCs""","""Co-culture""","""2.5D""","""Organoids""","""CD14+ cell-depleted allogeneic PBMCs""","""CTLs""","""CD4+ T-cells""","""Activation of monocytes""","""Low""","""Model development / theoretical""","""No""","""Supportive""","""Proof of Concept""","""Low""","""No""",2015
"""Alpha-1-antitrypsin deficiency""","""Immune cells profiling""","""Gene / Cell therapy""",null,"""Computational""","""Primary cell culture""","""Allogenic SCs""","""n/a""","""2D / 2.5D""","""Scaffolds""","""Daudi B lymphoblast cell line""","""Lymphocytes""","""Monocytes""","""Activation, proliferation and cytokines levels""","""Medium""","""Therapy development""","""Yes""",null,null,"""Medium""","""Yes""",2016
"""Broad""","""Immune response mechanism""","""Tissue engineering""",null,"""Mathematical""","""Primary cell culture / Immortalised cells""","""Amniotic mesenchymal cells (hAMTCs)""",null,"""3D""","""Spheroids""","""DCs""","""Macrophages""","""n/a""","""Allele and antigen mismatches""",null,"""Therapy development / mechanism of action""",null,null,null,null,null,2017
"""Cancer""","""Immunogenic properties""",null,null,null,"""Stem cells""","""AT-MSCs""",null,"""n/a""",null,"""Lymphocytes""","""Monocyte-derived DCs""",null,"""Antibody-dependent cellular cytotoxicity""",null,"""Therapy qualification""",null,null,null,null,null,2018
"""Cardiovascular""","""Immunogenicity decrease strategy""",null,null,null,null,"""BM-MSCs""",null,null,null,"""Macrophages""","""Monocytes""",null,"""CD107a, NK cell lytic activity""",null,null,null,null,null,null,null,2019
"""Cartilage disease""","""Immunotherapy""",null,null,null,null,"""BM-MSCs / AT-MSCs""",null,null,null,"""mixed""","""n/a""",null,"""CD107a, T cell proliferation""",null,null,null,null,null,null,null,null
"""Cholangiocarcinoma""","""Modulation of immune response""",null,null,null,null,"""BM-MSCs / Liver-MSCs""",null,null,null,"""Monocytes""","""NKs and DCs""",null,"""CD45RO, CD28, CCR7, CD95, proportion of different T cell phenotypes""",null,null,null,null,null,null,null,null
"""Diabetes""","""Regeneration strategy""",null,null,null,null,"""BM-MSCs derived chondrocytes / AT-MSC derived chondrocytes""",null,null,null,"""Monocyte-derived DCs""","""PBMCs""",null,"""CD8/CD4/reg T cell proliferation, CD3 T cell infiltration, TNF-α, CD25, CD69, PRF1, GZMB, FOXP3""",null,null,null,null,null,null,null,null


# Preparing test models

In [10]:
from getpaper.config import load_environment_keys
key = load_environment_keys()
test_atmp_models = atmp.extended_models.filter(pl.col("token_number") <= 10000)
test_atmp_models

environment found at /home/antonkulaga/sources/non-animal-models/.env


shape: (10, 31)
┌────────────┬────────────┬────────────┬─────────┬───┬───────────┬────────────┬────────────┬───────┐
│ Model      ┆ Disease    ┆ Study /    ┆ ATMP    ┆ … ┆ parsed_pa ┆ parsed_exi ┆ token_numb ┆ text  │
│ number     ┆ area       ┆ Disease    ┆ type    ┆   ┆ th        ┆ sts        ┆ er         ┆ ---   │
│ ---        ┆ ---        ┆ Feature    ┆ ---     ┆   ┆ ---       ┆ ---        ┆ ---        ┆ str   │
│ i64        ┆ str        ┆ ---        ┆ str     ┆   ┆ object    ┆ bool       ┆ i64        ┆       │
│            ┆            ┆ str        ┆         ┆   ┆           ┆            ┆            ┆       │
╞════════════╪════════════╪════════════╪═════════╪═══╪═══════════╪════════════╪════════════╪═══════╡
│ 9          ┆ Skeletal   ┆ Modulation ┆ Gene /  ┆ … ┆ /home/ant ┆ true       ┆ 8977       ┆ Brazi │
│            ┆            ┆ of immune  ┆ Cell    ┆   ┆ onkulaga/ ┆            ┆            ┆ lian  │
│            ┆            ┆ response   ┆ therapy ┆   ┆ sources/n ┆            ┆            ┆ Journ │
│            ┆            ┆            ┆         ┆   ┆ on-animal ┆            ┆            ┆ al of │
│            ┆            ┆            ┆         ┆   ┆ -models/d ┆            ┆            ┆ Medic │
│            ┆            ┆            ┆         ┆   ┆ ata/input ┆            ┆            ┆ al    │
│            ┆            ┆            ┆         ┆   ┆ s/dataset ┆            ┆            ┆ and   │
│            ┆            ┆            ┆         ┆   ┆ s/ATMP/pa ┆            ┆            ┆ Biolo │
│            ┆            ┆            ┆         ┆   ┆ rsed_pape ┆            ┆            ┆ gical │
│            ┆            ┆            ┆         ┆   ┆ rs/10.159 ┆            ┆            ┆ Resea │
│            ┆            ┆            ┆         ┆   ┆ 0/1414-43 ┆            ┆            ┆ rch   │
│            ┆            ┆            ┆         ┆   ┆ 1X2015420 ┆            ┆            ┆ (2015 │
│            ┆            ┆            ┆         ┆   ┆ 9.txt     ┆            ┆            ┆ ) 00( │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ 00):  │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ 1-8,  │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ http: │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ //dx. │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ doi.o │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ rg/10 │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ .1590 │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ /1414 │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ -431X │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ 20154 │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ 209   │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ ISSN  │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ 1809- │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ 452X  │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆       │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ CTLA4 │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ enhan │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ ces   │
│            ┆            ┆            ┆         ┆   ┆           ┆            ┆            ┆ the   │
│            ┆            ┆            ┆     

# Disease are classification

In [11]:
test_text = test_atmp_models.head().select("text").to_series()[0]
#test_text
diseases = atmp.dropdowns.select("Disease area").filter(pl.col("Disease area").is_not_null()).unique().to_series().to_list()
diseases

['AIDS',
 'Allotransplantation',
 'Alpha-1-antitrypsin deficiency',
 'Broad',
 'Cancer',
 'Cardiovascular',
 'Cartilage disease',
 'Cholangiocarcinoma',
 'Diabetes',
 'Ebola',
 'Eye diseases',
 'Immune disorders',
 'Immune oncology',
 'Inflammatory bowel disease',
 'Liver disease',
 'Melanoma',
 'Multiple Sclerosis',
 'Neurodegeneration',
 'Psoriasis',
 'Skeletal',
 'Skin',
 'Trauma',
 'Wound healing']

In [12]:
",".join(diseases)

'AIDS,Allotransplantation,Alpha-1-antitrypsin deficiency,Broad,Cancer,Cardiovascular,Cartilage disease,Cholangiocarcinoma,Diabetes,Ebola,Eye diseases,Immune disorders,Immune oncology,Inflammatory bowel disease,Liver disease,Melanoma,Multiple Sclerosis,Neurodegeneration,Psoriasis,Skeletal,Skin,Trauma,Wound healing'

In [13]:
from nam.wishes import DiseaseClassifier
d = DiseaseClassifier(values = diseases)
d

In [16]:
test_result = d.predict(test_text)
test_result

{'confidence': '90%', 'disease_area': 'Bone tissue engineering'}

In [ ]:
test_results = d.apply(test_atmp_models)

In [26]:
test_results

shape: (10, 6)
┌──────────────┬──────────────────┬──────────────┬──────────────────┬─────────────────┬────────────┐
│ Model number ┆ Disease area     ┆ token_number ┆ text             ┆ predicted_disea ┆ confidence │
│ ---          ┆ ---              ┆ ---          ┆ ---              ┆ se_area         ┆ ---        │
│ i64          ┆ str              ┆ i64          ┆ str              ┆ ---             ┆ str        │
│              ┆                  ┆              ┆                  ┆ str             ┆            │
╞══════════════╪══════════════════╪══════════════╪══════════════════╪═════════════════╪════════════╡
│ 9            ┆ Skeletal         ┆ 8977         ┆ Brazilian        ┆ Bone tissue     ┆ 90%        │
│              ┆                  ┆              ┆ Journal of       ┆ engineering     ┆            │
│              ┆                  ┆              ┆ Medical and      ┆                 ┆            │
│              ┆                  ┆              ┆ Biological       ┆                 ┆            │
│              ┆                  ┆              ┆ Research (2015)  ┆                 ┆            │
│              ┆                  ┆              ┆ 00(00): 1-8, htt ┆                 ┆            │
│              ┆                  ┆              ┆ p://dx.doi.org/1 ┆                 ┆            │
│              ┆                  ┆              ┆ 0.1590/1414-431X ┆                 ┆            │
│              ┆                  ┆              ┆ 20154209 ISSN    ┆                 ┆            │
│              ┆                  ┆              ┆ 1809-452X        ┆                 ┆            │
│              ┆                  ┆              ┆                  ┆                 ┆            │
│              ┆                  ┆              ┆ CTLA4 enhances   ┆                 ┆            │
│              ┆                  ┆              ┆ the osteogenic   ┆                 ┆            │
│              ┆                  ┆              ┆ differentiation  ┆                 ┆            │
│              ┆                  ┆              ┆ of allogeneic    ┆                 ┆            │
│              ┆                  ┆              ┆ human            ┆                 ┆            │
│              ┆                  ┆              ┆ mesenchymal stem ┆                 ┆            │
│              ┆                  ┆              ┆ cells in a model ┆                 ┆            │
│              ┆                  ┆              ┆ of immune        ┆                 ┆            │
│              ┆                  ┆              ┆ activation       ┆                 ┆            │
│              ┆                  ┆              ┆                  ┆                 ┆            │
│              ┆                  ┆              ┆ F. Dai1*, F.     ┆                 ┆            │
│              ┆                  ┆              ┆ Zhang1*, D.      ┆                 ┆            │
│              ┆                  ┆              ┆ Sun1, Z.H.       ┆                 ┆            │
│              ┆                  ┆              ┆ Zhang1, S.W.     ┆                 ┆            │
│              ┆                  ┆              ┆ Dong2 and J.Z.   ┆                 ┆            │
│              ┆                  ┆              ┆ Xu1              ┆                 ┆            │
│              ┆                  ┆              ┆                  ┆                 ┆            │
│              ┆                  ┆              ┆ 1National and    ┆                 ┆            │
│              ┆                  ┆              ┆ Regional United  ┆                 ┆            │
│              ┆                  ┆              ┆ Engineering      ┆                 ┆            │
│              ┆                  ┆              ┆ Laboratory of    ┆                 ┆            │
│              ┆                  ┆              ┆ Tissue           ┆                 ┆            │
│              ┆                  ┆           

In [30]:
test_results.drop("text").write_csv("~/Desktop/prototype.tsv", separator='\t')

In [29]:
test_results.columns

['Model number',
 'Disease area',
 'token_number',
 'text',
 'predicted_disease_area',
 'confidence']